<a href="https://colab.research.google.com/github/invoker-qqwrv/Beida_tensorflow/blob/main/4_4breakpoint_call_back.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 断点续训的意思是因为某些原因模型还没有训练完成就被中断，
# 下一次训练可以在上一次训练的基础上继续训练而不用从头开始；这种方式对于你那些训练时间很长的模型来说非常友好。
# 如果要进行断点续训，那么得满足两个条件：
# （1）本地保存了模型训练中的快照；（即断点数据保存）
# （2）可以通过读取快照恢复模型训练的现场环境。（断点数据恢复）
# 这两个操作都用到了tensorflow中的train.Saver类。

In [ ]:
import tensorflow as tf
import os

fashion=tf.keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test)=fashion.load_data()
x_train,x_test=x_train/255.0,x_test/255.0
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

checkpoint_save_path="content/drive/MyDrive/Colab Notebooks/fashion.ckpt"#存放模型的路径和文件名
# 因为生成ckpt文件的时候会同步生成索引表，所以要判断是不是有了索引表来判断是否已经保存过模型参数。
# 如果有课索引表就可以调用load_weights函数读取模型参数
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)
# 用tensorfolw的callback函数直接保存训练的模型参数
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,#文件存储路径
                      save_weights_only=True,#只保留模型参数
                      save_best_only=True)#只保留最新结果

history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])#加入callback选项return to history
model.summary()

4422102/4422102 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5014 - sparse_categorical_accuracy: 0.8249 - val_loss: 0.4208 - val_sparse_categorical_accuracy: 0.8485
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3724 - sparse_categorical_accuracy: 0.8656 - val_loss: 0.4000 - val_sparse_categorical_accuracy: 0.8551
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3354 - sparse_categorical_accuracy: 0.8779 - val_loss: 0.3719 - val_sparse_categorical_accuracy: 0.8670
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3104 - sparse_categorical_accuracy: 0.8862 - val_loss: 0.3566 - val_sparse_categorical_accuracy: 0.8720
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2935 - sparse_categorical_accuracy: 0.8919 - val_loss: 0.3667 - val_sparse_categorical_accuracy: 0.8660
Model: "sequential"
____________